Здесь я строю отображение номера категории в категории на более высоких уровнях иерархии

In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import LabelEncoder

In [2]:
categories = pd.read_csv('../data/category.csv', index_col='category_id')

In [3]:
cats = categories.name.str.split('|',expand=True)
n_levels = cats.shape[1]

In [4]:
print(n_levels)
cats

4


,0,1,2,3
category_id,,,,
0,Бытовая электроника,Телефоны,iPhone,None
1,Бытовая электроника,Ноутбуки,None,None
2,Бытовая электроника,Телефоны,Samsung,None
3,Бытовая электроника,Планшеты и электронные книги,Планшеты,None
4,Бытовая электроника,"Игры, приставки и программы",Игровые приставки,None
5,Бытовая электроника,Аудио и видео,Телевизоры и проекторы,None
6,Бытовая электроника,Телефоны,Другие марки,None
7,Бытовая электроника,Настольные компьютеры,None,None
8,Бытовая электроника,"Игры, приставки и программы",Игры для приставок,None


Чтобы получить уникальный лейбл для категории на каждом уровне, добавим к имени подкатегории названия всех ее над категорий и затем применим __LabelEncoder__. Так даже если у какой-то категории меньше уровней вложенности, то она все равно будет отличаться от остальных, хотя имя будет одинаковым на нескольких уровнях.

In [5]:
cats_copy = cats.copy()

le = LabelEncoder()
for  i in range(n_levels):
    
    lines = cats.loc[:, :n_levels-i-1].apply(lambda x: ' | '.join(x.astype(str)), axis=1)
    labels = le.fit_transform(lines)
    cats[n_levels-i-1] = labels

Посмотрим сколько различных категорий на каждом уровне иерархии

In [6]:
cats.apply(lambda x: len(np.unique(x)), axis=0)

0     4
1    23
2    51
3    54
dtype: int64

Нас не интересует последний уровень иерархии, поскольку он должен совпадать (и совпадает с точностью до перестановки) с __category_id__. 

Построим словари конвертирующие __category_id__ в лейблы на более высоких уровнях вложености.

In [7]:
label_converter = {
    level: {
        row[0]: row[1]
        for row in cats[[level]].itertuples()
    }
    
    for level in range(3)      
}

with open('../data/label_converter.json', 'w') as f:
    
    json.dump(label_converter, f)